<a href="https://colab.research.google.com/github/EloizioHMD/Python_DS/blob/main/scrapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Scrapping



Beautiful Soup é uma biblioteca que facilita a extração de informações de páginas da web. Ele fica no topo de um analisador HTML ou XML, fornecendo idiomas Pythonic para iterar, pesquisar e modificar a árvore de análise.
Para mais informações consultar a biblioteca do [BeautifulSoup]('https://www.crummy.com/software/BeautifulSoup/bs4/doc/').

In [ ]:
## Bibliotecas para o projeto
import pandas as pd
from bs4 import BeautifulSoup
import requests

## Url e User Agents

In [ ]:
url = 'https://www.vivareal.com.br/venda/pernambuco/recife/apartamento_residencial/?pagina={npagina}' # {npagina} é a iteração para mudar as páginas

userAgents = [ 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36' ] # Identificando-se como Chrome no Windows 10

## Análise dos elementos do Site

Este trecho serve para testarmos a extração de cada elemento desejado, seja nome do imóvel, preço etc. Quando compreendido, movemos o trecho de código finalizado para o bloco mais adiante no notebook, em que o scrapping é feito em lote.

### Passo 01 : Capturar o HTML com requests


In [ ]:
doc = requests.get( url.format(npagina=1), headers={'User-agent':userAgents[0]} )

Vamos verificar o que o webserver retornou olhando só os 700 primeiros caracteres (variavel olhadinha).

In [ ]:
doc.content[:700]

b' <!DOCTYPE html> <html lang="pt-BR" prefix="og: https://ogp.me/ns#"> <head> <meta name="msvalidate.01" content="5DB2670D7BB1D1D3E36C46F7C3D59380"> <meta name="omniverify" content="omni2c019e0"> <meta property="og:url" content="https://www.vivareal.com.br/venda/pernambuco/recife/apartamento_residencial/"> <meta property="og:type" content=""> <meta property="og:image" content=""> <meta property="og:image:type" content="image/jpeg"> <meta property="og:image:secure_url" content="">  <meta property="og:title" content="Apartamentos \xc3\xa0 venda em Recife, PE - Viva Real"> <meta property="og:description" content="Mais de 43.717 apartamentos \xc3\xa0 venda em Recife, Pernambuco. Acesse as melhores ofertas de '

### Passo 02 : Analisando o HTML

In [ ]:
analisador = BeautifulSoup(doc.content, 'html.parser')

In [ ]:
analisador

 <!DOCTYPE html>
 <html lang="pt-BR" prefix="og: https://ogp.me/ns#"> <head> <meta content="5DB2670D7BB1D1D3E36C46F7C3D59380" name="msvalidate.01"/> <meta content="omni2c019e0" name="omniverify"/> <meta content="https://www.vivareal.com.br/venda/pernambuco/recife/apartamento_residencial/" property="og:url"/> <meta content="" property="og:type"/> <meta content="" property="og:image"/> <meta content="image/jpeg" property="og:image:type"/> <meta content="" property="og:image:secure_url"/> <meta content="Apartamentos à venda em Recife, PE - Viva Real" property="og:title"/> <meta content="Mais de 43.717 apartamentos à venda em Recife, Pernambuco. Acesse as melhores ofertas de venda de apartamentos em Recife." property="og:description"/> <meta content="Mais de 43.717 apartamentos à venda em Recife, Pernambuco. Acesse as melhores ofertas de venda de apartamentos em Recife." name="description"/> <title> Apartamentos à venda em Recife, PE - Viva Real </title> <link href="https://www.vivareal.co

Nesse ponto é interessante partir para observar as tags com a ferramenta de desenvolvedor do navegar para saber que informações buscar.

In [ ]:
imoveis = analisador.find_all('div', class_='js-card-selector')

In [ ]:
len(imoveis)

36

In [ ]:
imoveis[0]

<div class="js-card-selector"> <article class="property-card__container js-property-card" data-see-phone=""> <div class="property-card__main-info"> <div class="property-card__main-link"> <div class="property-card__carousel js-property-carousel "> <a class="property-card__labels-container js-main-info js-listing-labels-link" href="/imovel/apartamento-2-quartos-varzea-bairros-recife-com-garagem-55m2-venda-RS330000-id-2558022053/"> <div class="property-card__inactive-listing">Indisponível</div> <div class="property-card__already-seen">Visualizado</div> </a> <div class="carousel__container js-carousel-scroll" style="width: calc(4 * 100%);"> <div class="carousel__item-wrapper js-carousel-item-wrapper"> <img alt="Foto 1 de Apartamento com 2 Quartos à venda, 55m² em Várzea, Recife" class="carousel__image js-carousel-image" src="https://resizedimgs.vivareal.com/crop/286x200/named.images.sp/6ae2a74c322a18e44b32a90d0e0b6440/foto-1-de-apartamento-com-2-quartos-a-venda-55m-em-varzea-recife.jpg" ti

### Passo 3 : Montar estrutura para extrair partes desejadas

Estrutura para pegar o endereço do imóvel:

In [ ]:
imoveis[0].find( 'span', class_='property-card__address' ).text.strip()

'Rua General Polidoro, 306 - Várzea, Recife - PE'

Estrutura para extair do valor anterior bairro e endereço.

In [ ]:
end_str = imoveis[0].find( 'span', class_='property-card__address' ).text.strip()
split01 = end_str.split(' - ')
end = split01[0].split(',')
bairro = split01[1].split(',')
end[0]

'Rua General Polidoro'

In [ ]:
end_str = imoveis[0].find( 'span', class_='property-card__address' ).text.strip()
split01 = end_str.split(' - ')
bairro = split01[1].split(',')
print('Bairro é: ' + str(bairro[0]))

Bairro é: Várzea


Estrurura para pegar o tamanho do imóvel em m²:

In [ ]:
imoveis[0].find( 'span', class_='property-card__detail-value js-property-card-value property-card__detail-area js-property-card-detail-area').text.strip()

'55'

Estrutura de quantidade de quartos:

In [ ]:
imoveis[0].find( 'li', class_='property-card__detail-item property-card__detail-room js-property-detail-rooms').text.replace('Quartos','').strip()

'2'

Estrutura de quantidade de banheiro:

In [ ]:
imoveis[0].find( 'li', class_='property-card__detail-item property-card__detail-bathroom js-property-detail-bathroom').text.replace('Banheiros','').strip()

'2'

Estrutura de quantidade de vagas:

In [ ]:
imoveis[0].find( 'li', class_='property-card__detail-item property-card__detail-garage js-property-detail-garages').text.replace('Vaga','').strip()

'1'

Estrutura do valor do imóvel com `replace` para apenas o valor:

In [ ]:
imoveis[0].find( 'div', class_='property-card__price js-property-card-prices js-property-card__price-small').text.replace('R$', '').replace('.', '').strip()

'330000'

Estrutura do valor do condomínio: 

In [ ]:
imoveis[0].find( 'strong', class_='js-condo-price' ).text.replace('R$', '').replace('.', '').strip()

'350'

### Passo 4 : Estrutura de Repetição em massa

In [ ]:
paginas_test = 1190

In [ ]:
fichas = []

for paginas_test in range(1, paginas_test):
  print('Pagina', url.format(npagina=paginas_test))

  doc = requests.get( url.format(npagina=paginas_test), headers={'User-agent':userAgents[0]} )
  analisador = BeautifulSoup(doc.content, 'html.parser')

  imoveis = analisador.find_all('div', class_='js-card-selector')

  for unidade in imoveis:
    uni = {}

    end_str = unidade.find( 'span', class_='property-card__address' ).text.strip()
    split01 = end_str.split(' - ')
    end = split01[0].split(',')
    bairro = split01[1].split(',')
    uni['endereco'] = end[0]
    uni['bairro']  = bairro[0]
    uni['area_m2'] = unidade.find( 'span', class_='property-card__detail-value js-property-card-value property-card__detail-area js-property-card-detail-area').text.strip()
    uni['quartos'] = unidade.find( 'li', class_='property-card__detail-item property-card__detail-room js-property-detail-rooms').text.replace('Quartos','').strip()
    uni['banheiros'] = unidade.find( 'li', class_='property-card__detail-item property-card__detail-bathroom js-property-detail-bathroom').text.replace('Banheiros','').strip()
    uni['vagas'] = unidade.find( 'li', class_='property-card__detail-item property-card__detail-garage js-property-detail-garages').text.replace('Vaga','').strip()
    try:
      uni['valor'] = unidade.find( 'div', class_='property-card__price js-property-card-prices js-property-card__price-small').text.replace('R$', '').replace('.', '').strip()
    except:
      uni['valor'] = 'n'
    try:
      uni['condominio'] = unidade.find( 'strong', class_='js-condo-price' ).text.replace('R$', '').replace('.', '').strip()
    except:
      uni['condominio'] = 'n'
    fichas.append(uni) 

Pagina https://www.vivareal.com.br/venda/pernambuco/recife/apartamento_residencial/?pagina=1
Pagina https://www.vivareal.com.br/venda/pernambuco/recife/apartamento_residencial/?pagina=2
Pagina https://www.vivareal.com.br/venda/pernambuco/recife/apartamento_residencial/?pagina=3
Pagina https://www.vivareal.com.br/venda/pernambuco/recife/apartamento_residencial/?pagina=4
Pagina https://www.vivareal.com.br/venda/pernambuco/recife/apartamento_residencial/?pagina=5
Pagina https://www.vivareal.com.br/venda/pernambuco/recife/apartamento_residencial/?pagina=6
Pagina https://www.vivareal.com.br/venda/pernambuco/recife/apartamento_residencial/?pagina=7
Pagina https://www.vivareal.com.br/venda/pernambuco/recife/apartamento_residencial/?pagina=8
Pagina https://www.vivareal.com.br/venda/pernambuco/recife/apartamento_residencial/?pagina=9
Pagina https://www.vivareal.com.br/venda/pernambuco/recife/apartamento_residencial/?pagina=10
Pagina https://www.vivareal.com.br/venda/pernambuco/recife/apartament

In [ ]:
fichas

[{'endereco': 'Rua General Polidoro',
  'bairro': 'Várzea',
  'area_m2': '55',
  'quartos': '2',
  'banheiros': '2',
  'vagas': '1',
  'valor': '330000',
  'condominio': '350'},
 {'endereco': 'Rua Monte Castelo',
  'bairro': 'Boa Vista',
  'area_m2': '33',
  'quartos': '1   Quarto',
  'banheiros': '1   Banheiro',
  'vagas': '1',
  'valor': 'Sob Consulta',
  'condominio': 'n'},
 {'endereco': 'Rua Setúbal',
  'bairro': 'Boa Viagem',
  'area_m2': '86',
  'quartos': '3',
  'banheiros': '3',
  'vagas': '2   s',
  'valor': '780000',
  'condominio': '740'},
 {'endereco': 'Rua Quarenta e Oito',
  'bairro': 'Aflitos',
  'area_m2': '69',
  'quartos': '3',
  'banheiros': '3',
  'vagas': '1',
  'valor': '378000',
  'condominio': '690'},
 {'endereco': 'Rua Esmeraldino Bandeira',
  'bairro': 'Graças',
  'area_m2': '54',
  'quartos': '1   Quarto',
  'banheiros': '1   Banheiro',
  'vagas': '1',
  'valor': '240000',
  'condominio': '590'},
 {'endereco': 'Rua Edson Álvares',
  'bairro': 'Casa Forte',
  

## Criar um Dataframe com as fichas

Criando a estrutura do dataframe paratir do uni.

In [ ]:
df = pd.DataFrame(columns=uni.keys())
df

,endereco,bairro,area_m2,quartos,banheiros,vagas,valor,condominio


Realizar o append das fichas para os endereços.

In [ ]:
df = df.append(fichas)
df.tail()

,endereco,bairro,area_m2,quartos,banheiros,vagas,valor,condominio
42799,Rua Visconde de Jequitinhonha,Boa Viagem,64,3,2,2 s,600000,n
42800,Rua do Riachuelo,Boa Vista,95,4,3,1,265000,555
42801,Rua Jonathas de Vasconcelos,Boa Viagem,55,2,2,1,305000,600
42802,Rua Leon Helmer,Boa Viagem,90,3,3,1,499999,850
42803,Rua Padre Landim,Madalena,107,4,3,1,430000,n


In [ ]:
df.to_csv('rec_imovel_02.csv', encoding = 'utf-8')